## Training on the WaDi Data Set

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from statsmodels.graphics.tsaplots import plot_acf
import math
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
# import torch
# from torch.utils.data import Dataset, DataLoader, TensorDataset
import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
import random
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from tabulate import tabulate

2026-01-09 10:08:52.324872: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
Train_df = pd.read_csv("/home/brandon/data/WADI_14days_new.csv")
Train_df = Train_df.drop(columns = ['Row'])
Train_df['Date'] = pd.to_datetime(Train_df['Date'], format='mixed')
Train_df['Attack_Label'] = np.zeros(Train_df.shape[0], dtype='float32')
Train_df

,Date,Time,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,...,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW,Attack_Label
0,2017-09-25,00:00.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,...,1,1,1,1,1,1,67.9651,1,0.68,0.0
1,2017-09-25,00:01.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,...,1,1,1,1,1,1,67.9651,1,0.68,0.0
2,2017-09-25,00:02.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,...,1,1,1,1,1,1,67.9651,1,0.68,0.0
3,2017-09-25,00:03.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,...,1,1,1,1,1,1,67.1948,1,0.68,0.0
4,2017-09-25,00:04.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,...,1,1,1,1,1,1,67.1948,1,0.68,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784566,2017-10-07,16:06.0,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,0,...,1,1,1,1,1,1,60.6305,1,0.25,0.0
784567,2017-10-07,16:07.0,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,0,...,1,1,1,1,1,1,60.6305,1,0.25,0.0
784568,2017-10-07,16:08.0,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,0,...,1,1,1,1,1,1,60.6305,1,0.25,0.0
784569,2017-10-07,16:09.0,175.896,0.613476,11.8913,479.224,0.331622,0.001173,0,0,...,1,1,1,1,1,1,60.4477,1,0.25,0.0


In [4]:
attack_df = pd.read_csv("/home/brandon/data/WADI_attackdataLABLE.csv", header = 1)
attack_df['Date'] = attack_df['Date ']
attack_df = attack_df[attack_df['Row '] <= 172801.0]
attack_df['Attack_Label'] = attack_df['Attack LABLE (1:No Attack, -1:Attack)']
attack_df = attack_df.drop(columns = ['Row ', 'Date ', 'Attack LABLE (1:No Attack, -1:Attack)'])
# attack_df['Date'] = pd.to_datetime(attack_df['Date'], format='%m/%d/%y')
attack_df

,Time,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,...,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW,Date,Attack_Label
0,00:00.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
1,00:01.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
2,00:02.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
3,00:03.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
4,00:04.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172796,59:56.0,172.959,0.547483,11.9184,466.034,0.318217,0.001222,0.0,0.0,55.5587,...,1.0,1.0,1.0,1.0,1.0,59.3546,1.0,0.00,10/11/17,1
172797,59:57.0,172.959,0.547483,11.9184,466.034,0.318217,0.001222,0.0,0.0,55.5587,...,1.0,1.0,1.0,1.0,1.0,59.3546,1.0,0.00,10/11/17,1
172798,59:58.0,172.915,0.583479,11.9211,466.051,0.318317,0.001260,0.0,0.0,55.7260,...,1.0,1.0,1.0,1.0,1.0,58.8102,1.0,0.00,10/11/17,1
172799,59:59.0,172.915,0.583479,11.9211,466.051,0.318317,0.001260,0.0,0.0,55.7260,...,1.0,1.0,1.0,1.0,1.0,58.8102,1.0,0.00,10/11/17,1


In [ ]:
Val_df = attack_df.loc[attack_df['Date'] == '10/9/17'].copy()

# 3. Create the Test DataFrame (Test_df) as the remainder
# Test_df = attack_df.loc[attack_df['Date'] != '10/9/17'].copy()

#Free up memory after splitting val and test sets
del attack_df


In [7]:
# Drop columns that are always NaN
# Identify columns that are always NaN in the training set
always_nan_cols = Train_df.columns[Train_df.isnull().all()].tolist()

print(f"\nNumber of columns that are always NaN in training data: {len(always_nan_cols)}")
print("Always-NaN columns:")
print(always_nan_cols)

Train_df = Train_df.drop(columns=always_nan_cols)
Val_df   = Val_df.drop(columns=always_nan_cols)
# Test_df  = Test_df.drop(columns=always_nan_cols)

print("\nAfter dropping always-NaN columns:")
print(f"Train shape: {Train_df.shape}")
print(f"Val shape:   {Val_df.shape}")
# print(f"Test shape:  {Test_df.shape}")


Number of columns that are always NaN in training data: 4
Always-NaN columns:
['2_LS_001_AL', '2_LS_002_AL', '2_P_001_STATUS', '2_P_002_STATUS']

After dropping always-NaN columns:
Train shape: (784571, 126)
Val shape:   (21600, 126)


In [8]:
# Use appropriate methods to handle missing values
# Use liner interpolation for missing values that spread out across the data
Train_df['1_AIT_002_PV'] = Train_df['1_AIT_002_PV'].interpolate(method='linear')
Train_df['1_AIT_004_PV'] = Train_df['1_AIT_004_PV'].interpolate(method='linear')
Train_df['2B_AIT_004_PV'] = Train_df['2B_AIT_004_PV'].interpolate(method='linear')
# Use forward fill for missing values that occur in blocks
Train_df['3_AIT_004_PV'] = Train_df['3_AIT_004_PV'].ffill()

In [9]:
# 4a. Find features using Train_df (as before)
features_original = [c for c in Train_df.columns if c not in ['Date', 'Time']]
constant_features = [col for col in features_original if Train_df[col].nunique() == 1]
features_final = [col for col in features_original if col not in constant_features]
print(f"Number of constant features removed: {len(constant_features)}")

scaler = MinMaxScaler()

# Transform the data using the NumPy arrays X_train, X_val, X_test
Train_scaled = scaler.fit_transform(Train_df[features_final])
Val_scaled = scaler.transform(Val_df[features_final])
# Test_scaled = scaler.transform(Test_df[features_final])

y_train = Train_df['Attack_Label'].values.astype('float32')
y_val = Val_df['Attack_Label'].values.astype('float32')
# y_test = Test_df['Attack_Label'].values.astype('float32')

Number of constant features removed: 31


In [12]:
def make_window_dataset(
    data,
    window_size,
    stride=1,
    batch_size=64
):
    ds = tf.data.Dataset.from_tensor_slices(data)
    ds = ds.window(
        size=window_size,
        shift=stride,
        drop_remainder=True
    )

    ds = ds.flat_map(lambda w: w.batch(window_size))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    # Autoencoder objective: X → X
    ds = ds.map(lambda x: (x, x))

    return ds


In [15]:
WINDOW_SIZE = 100
STRIDE = 1
BATCH_SIZE = 64
NUM_FEATURES = len(features_final)

train_ds = make_window_dataset(
    Train_scaled,
    window_size=WINDOW_SIZE,
    stride=STRIDE,
    batch_size=BATCH_SIZE
)

val_ds = make_window_dataset(
    Val_scaled,
    window_size=WINDOW_SIZE,
    stride=STRIDE,
    batch_size=BATCH_SIZE
)


In [16]:
#GRU
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'
tf.config.experimental.enable_op_determinism()

GRU_model = GRU_model = keras.Sequential([
    keras.Input(shape=(WINDOW_SIZE, NUM_FEATURES)),

    layers.GRU(units=32, return_sequences=False),
    layers.RepeatVector(WINDOW_SIZE),

    layers.GRU(units=32, return_sequences=True),
    layers.TimeDistributed(layers.Dense(NUM_FEATURES))
])

optimizer = keras.optimizers.Adam(learning_rate=1e-3)
GRU_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

GRU_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 32)             │        12,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 100, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100, 32)        │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 93)        │         3,069 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,597 (84.36 KB)

 Trainable params: 21,597 (84.36 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Train the GRU
GRU_history = GRU_model.fit(
    train_ds,
    validation_data = (val_ds),
    epochs = 30,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights = True)
    ]
)

Epoch 1/30


2026-01-09 10:26:27.472566: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91700


  12257/Unknown 197s 16ms/step - loss: 0.0168 - mae: 0.0692

2026-01-09 10:29:42.310667: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2026-01-09 10:29:42.310696: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/Shape/_6]]
/home/brandon/miniconda3/envs/ml/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


12258/12258 ━━━━━━━━━━━━━━━━━━━━ 201s 16ms/step - loss: 0.0145 - mae: 0.0641 - val_loss: 0.0332 - val_mae: 0.1103
Epoch 2/30
    7/12258 ━━━━━━━━━━━━━━━━━━━━ 3:28 17ms/step - loss: 0.0319 - mae: 0.1103

...

12258/12258 ━━━━━━━━━━━━━━━━━━━━ 198s 16ms/step - loss: 0.0137 - mae: 0.0615 - val_loss: 0.0205 - val_mae: 0.0845
Epoch 21/30
12255/12258 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0129 - mae: 0.0593

2026-01-09 11:35:50.228181: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 13410228638603860364
2026-01-09 11:35:50.228208: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10084876986140428299
2026-01-09 11:35:50.228214: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 17767549733324248299


12258/12258 ━━━━━━━━━━━━━━━━━━━━ 198s 16ms/step - loss: 0.0130 - mae: 0.0602 - val_loss: 0.0196 - val_mae: 0.0828
Epoch 22/30
12255/12258 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0126 - mae: 0.0585

2026-01-09 11:39:08.673394: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 17767549733324248299


12258/12258 ━━━━━━━━━━━━━━━━━━━━ 198s 16ms/step - loss: 0.0126 - mae: 0.0591 - val_loss: 0.0189 - val_mae: 0.0817
Epoch 23/30
12258/12258 ━━━━━━━━━━━━━━━━━━━━ 198s 16ms/step - loss: 0.0127 - mae: 0.0586 - val_loss: 0.0188 - val_mae: 0.0858
Epoch 24/30
12258/12258 ━━━━━━━━━━━━━━━━━━━━ 199s 16ms/step - loss: 0.0137 - mae: 0.0617 - val_loss: 0.0145 - val_mae: 0.0777


In [ ]:
# Run GRU predictions
GRU_X_val_pred = GRU_model.predict(X_val)

# X_val is your validation_windows tensor
X_val_np = X_val.numpy()

GRU_y_val_np = y_val_window_labels.numpy()

# Compute reconstruction error per sequence
GRU_reconstruction_error = np.mean(np.square(GRU_X_val_pred - X_val_np), axis=(1, 2))

GRU_BEST_THRESHOLD = 0.20

GRU_y_pred_labels = (GRU_reconstruction_error > GRU_BEST_THRESHOLD).astype(int)
GRU_prec, GRU_rec, GRU_f1, _ = precision_recall_fscore_support(
    GRU_y_val_np , GRU_y_pred_labels, average='binary', zero_division=0
)

# Calculate mean and std for reconstruction errors for tuning.
GRU_mean_error = np.mean(GRU_reconstruction_error)
GRU_std_error = np.std(GRU_reconstruction_error)
GRU_training_error = GRU_history.history['loss'][-1]
GRU_validation_error = GRU_history.history['val_loss'][-1]

print(f"GRU Metrics: Mean Error = {GRU_mean_error:.4f}, Std Dev = {GRU_std_error:.4f}, Training Error = {GRU_training_error:.4f}, Validation Error = {GRU_validation_error:.4f}")
print(f"Precision: {GRU_prec:.3f} | Recall: {GRU_rec:.3f} | F1: {GRU_f1:.3f}")

In [ ]:
loss_key = 'loss'
val_loss_key = 'val_loss'

plt.figure(figsize=(8,5))
plt.plot(GRU_history.history[loss_key], label='Training Loss')
plt.plot(GRU_history.history[val_loss_key], label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training vs Validation Loss (Transformer Autoencoder)")
plt.legend()
plt.grid(True)
plt.show()